### Import Required Libraries and Set Up Environment Variables

In [18]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [19]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [13]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# Use a sort filter, sort by newest
sort = "newest"
# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"
# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"
# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}")

In [20]:
# create a list to store the reviews
reviews_list = ([])  
# Attempt to retrieve 200 reviews by iterating through 20 pages starting at 0 (10 results per page)
for page in range(2):  # NYT API limits results to 10 per page, 20 pages = 200 results
    offset = page * 2
    full_url = f"{url}api-key={nyt_api_key}&q=love&fq={filter_query}&begin_date={begin_date}&end_date={end_date}&sort={sort}&fl={field_list}&page={page}"
    
    response = requests.get(full_url)
    # Make a "GET" requyest and retrieve the JSON
    # reviews = requests.get(url).json()
# print(json.dumps(reviews, indent=4))
    time.sleep(6)  #  rate limits per documentation

    try:
        reviews = response.json()
        if reviews["response"]["docs"]:
            reviews_list.extend(reviews["response"]["docs"])
            print(f"Page {page + 1} processed, offset {offset}")
        else:
            print(f"No more results found at page {page + 1}, stopping.")
            break
    except Exception as e:
        print(f"An error occurred on page {page + 1} with offset {offset}: {e}")
        break

An error occurred on page 1 with offset 0: 'response'


In [17]:
#Coonfirm 200 reviews were retrieved
print(len(reviews_list))

# Preview the first 5 results in JSON format
first_five = reviews_list[:5]
# Use json.dumps with argument indent=4 to format data
json_data = json.dumps(first_five, indent=4)
print(json_data)

0
[]


In [32]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)

# Optionally, preview the first few rows of the DataFrame
#print(reviews_df.head(3))
reviews_df.head(3) 

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None


In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# End string should include " Review" to avoid cutting title early
# Using "\u2019 Review" to find the end of the title does not work too well because often the string in "headline.main" does not contain "\u2019 Review".
reviews_df["title"] = reviews_df["headline.main"].apply(
    lambda st: st[st.find("\u2018") + 1 : st.find("\u2019 ")]
)
# Sometimes we end up with a trailing comma. Remove it.
reviews_df["title"] = reviews_df["title"].str.rstrip(",")

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};"
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)

In [30]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df["title"].to_list()

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_api_key = "&api_key=" + tmdb_api_key


TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
# Create an empty list to store the results
import random
import time
tmdb_movies_list = []   

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter= 1

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0:
        # Sleep for a random time between 1 and 5 seconds
        time.sleep(1)
        print("Sleeping for 1 second")

    # Add 1 to the request counter
    request_counter += 1    

    # Perform a "GET" request for The Movie Database
    tmdb_response = requests.get(url + title + tmdb_key_string) .json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = tmdb_response["results"][0]["id"]

        # Make a request for a the full movie details
        movie_details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        tmdb_response_details = requests.get(movie_details_url).json()

        # Extract the genre names into a list
        genres = [tmdb_response_details['genres'][i]['name'] for i in range(len(tmdb_response_details['genres']))]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [tmdb_response_details['spoken_languages'][i]['english_name'] for i in range(len(tmdb_response_details['spoken_languages']))]

        # Extract the production_countries' name into a list
        production_countries = [tmdb_response_details['production_countries'][i]['name'] for i in range(len(tmdb_response_details['production_countries']))]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        dict_movie_details = {'title': title,
                              'original_title': tmdb_response_details['original_title'],
                              'budget': tmdb_response_details['budget'],
                              'original_language': tmdb_response_details['original_language'],
                              'homepage': tmdb_response_details['homepage'],
                              'overview': tmdb_response_details['overview'],
                              'popularity': tmdb_response_details['popularity'],
                              'runtime': tmdb_response_details['runtime'],
                              'revenue': tmdb_response_details['revenue'],
                              'release_date': tmdb_response_details['release_date'],
                              'vote_average': tmdb_response_details['vote_average'],
                              'vote_count': tmdb_response_details['vote_count'],
                              'genres': genres,
                              'spoken_languages': spoken_languages,
                              'production_countries': production_countries}
        tmdb_movies_list.append(dict_movie_details)

        # Print out the title that was found
        print(f'The movie called "{title}" was found.')
    except:
        print(f'NOT FOUND: "{title}".')

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

In [ ]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.head(5)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_movies_df, reviews_df, on="title")

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_fix = ['genres', 'spoken_language', 'production_countries',]  

# Create a list of characters to remove
characters_remove = ['[', ']', '"']

# Loop through the list of columns to fix
for col in columns_fix:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
    for char in characters_remove:
        # Remove the character
        merged_df[col] = merged_df[col].str.replace(char, '')

# Display the fixed DataFrame
display(merged_df.head())

In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])

In [ ]:
# Delete duplicate rows and reset index
print(merged_df.shape)
merged_df = merged_df.drop_duplicates(inplace=True)
print(merged_df.shape)
merged_df.reset_index(drop=True, inplace=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv('output/movie_data.csv', index=False)